# 🤖 Tutorial de Agentes ADK (Agent Developer Kit) - Español

Este notebook es una introducción práctica al desarrollo de agentes inteligentes usando el Agent Developer Kit (ADK) de Google con modelos Gemini.


## 1. Instalación y Configuración

### 1.1 Instalar las Dependencias Necesarias

Primero, instalamos el Agent Developer Kit y configuramos las claves de API necesarias.

In [5]:
%pip install -U -q google-adk

### 1.2 Importar las Librerías Requeridas

In [25]:
# Importación de las librerías necesarias
from google.adk.agents.llm_agent import LlmAgent
from google.adk.runners import Runner
from google.adk.tools import google_search
from google.adk.code_executors import BuiltInCodeExecutor
from google.adk.sessions import InMemorySessionService
from pydantic import BaseModel, Field
from typing import List, Dict, Optional
from google.genai import types

import os
import asyncio

### 1.3 Configurar las Variables de Entorno

Es importante configurar las claves API de manera segura usando variables de entorno.

In [11]:
import os

PROJECT_ID = "gde-access"  # @param {type: "string", placeholder: "[your-project-id]", isTemplate: true}
if not PROJECT_ID or PROJECT_ID == "[your-project-id]":
    PROJECT_ID = str(os.environ.get("GOOGLE_CLOUD_PROJECT"))

LOCATION = "global"

In [12]:
os.environ["GOOGLE_CLOUD_PROJECT"] = PROJECT_ID
os.environ["GOOGLE_CLOUD_LOCATION"] = LOCATION
os.environ["GOOGLE_GENAI_USE_VERTEXAI"] = "TRUE"  # fuerza ADK/google-genai a usar Vertex

### 1.4 Inicializar el Modelo Gemini

Creamos una instancia del modelo Gemini que será usado por nuestros agentes.

In [14]:
# Crear instancia del modelo Gemini 2.0 Flash
MODEL_GEMINI = "gemini-2.5-flash"

print(f"✅ Modelo inicializado: {MODEL_GEMINI}")

✅ Modelo inicializado: gemini-2.5-flash


### 2. Arquitectura del ADK

```
┌─────────────────────────────────────────────┐
│              Google ADK                     │
├─────────────────────────────────────────────┤
│                                             │
│  ┌─────────┐  ┌─────────┐  ┌─────────┐      │
│  │ Agentes │  │  Tools  │  │Sessions │      │
│  └────┬────┘  └────┬────┘  └────┬────┘      │
│       │            │             │          │
│  ┌────┴────────────┴─────────────┴────┐     │
│  │           Ejecutores (Runners)     │     │
│  └────────────────────────────────────┘     │
│                                             │
└─────────────────────────────────────────────┘
```

### Componentes Clave:

1. **🤖 Agentes (Agents)**
   - `LlmAgent`: Agente impulsado por LLM
   - `WorkflowAgent`: Orquestador de otros agentes
   - Tipos especializados: Sequential, Parallel, Loop

2. **🔧 Herramientas (Tools)**
   - Funciones que los agentes pueden usar
   - Preconstruidas: búsqueda, código, etc.
   - Personalizables según necesidades

3. **▶️ Ejecutores (Runners)**
   - Gestionan el flujo de ejecución
   - Manejan mensajes y eventos
   - Controlan el estado

4. **💾 Sesiones (Sessions)**
   - Mantienen contexto entre interacciones
   - Persisten información importante
   - Habilitan conversaciones continuas

## 3. Creación de tu Primer Agente

### 3.1 Agente Básico

Vamos a crear un agente simple que puede responder preguntas.

In [15]:
# Crear un agente básico
mi_primer_agente = LlmAgent(
    model=MODEL_GEMINI,
    name="asistente_basico",
    description="Un asistente útil que puede responder preguntas generales",
    instruction="""Eres un asistente amigable y útil.
    Responde las preguntas de manera clara y concisa.
    Si no sabes algo, admítelo honestamente."""
)

print("🤖 Agente creado exitosamente!")
print(f"Nombre: {mi_primer_agente.name}")
print(f"Descripción: {mi_primer_agente.description}")

🤖 Agente creado exitosamente!
Nombre: asistente_basico
Descripción: Un asistente útil que puede responder preguntas generales


### 3.2 Ejecutar el Agente de Forma Síncrona

In [32]:
# --- Gestión de Sesiones ---
# Concepto Clave: SessionService almacena el historial de la conversación y el estado.
# InMemorySessionService es un almacenamiento simple y no persistente para este tutorial.
session_service = InMemorySessionService()

# Definir constantes para identificar el contexto de la interacción
APP_NAME = "demo"
USER_ID = "user_1"
SESSION_ID = "session_001" # Usando un ID fijo por simplicidad

# Crear la sesión específica donde ocurrirá la conversación
session = await session_service.create_session(
    app_name=APP_NAME,
    user_id=USER_ID,
    session_id=SESSION_ID
)
print(f"Sesión creada: App='{APP_NAME}', Usuario='{USER_ID}', Sesión='{SESSION_ID}'")

Sesión creada: App='demo', Usuario='user_1', Sesión='session_001'


In [33]:
# Runner: This is the main component that manages the interaction with the agent.
runner = Runner(agent=mi_primer_agente,
                app_name=APP_NAME,
                session_service=session_service)

print(f"Runner creado para el agente '{runner.agent.name}'.")

Runner creado para el agente 'asistente_basico'.


In [35]:
events = runner.run(user_id=USER_ID,
          session_id=SESSION_ID,
          new_message=types.Content(role='user', parts=[types.Part(text="Cual es la capital de Colombia?")]))

for event in events:
    if event.is_final_response():
        final_response = event.content.parts[0].text
        print("Agent Response: ", final_response)

Agent Response:  La capital de Colombia es **Bogotá**.


### 3.3 Ejecutar el Agente de Forma Asíncrona


### Comunicación con el agente usando `async`

Necesitamos una forma de enviar mensajes a nuestro agente y recibir sus respuestas. Dado que las llamadas a modelos de lenguaje (LLMs) y la ejecución de herramientas pueden tomar tiempo, el `Runner` del ADK funciona de manera asíncrona.

Vamos a definir una función auxiliar asíncrona (`call_agent_async`) que:

* Recibe una cadena de texto con la consulta del usuario.
* La empaqueta en el formato `Content` del ADK.
* Llama a `runner.run_async`, proporcionando el contexto del usuario/sesión y el nuevo mensaje.
* Itera a través de los `Events` generados por el `runner`. Los eventos representan pasos en la ejecución del agente (por ejemplo: solicitud de herramienta, recepción del resultado, pensamiento intermedio del LLM, respuesta final).
* Identifica e imprime el evento de respuesta final usando `event.is_final_response()`.

#### ¿Por qué usar `async`?

Las interacciones con LLMs y herramientas (como APIs externas) son operaciones dependientes de entrada/salida (I/O-bound). Usar `asyncio` permite manejar estas operaciones de forma eficiente sin bloquear la ejecución del programa.

---


In [28]:
async def call_agent_async(query: str, runner, user_id, session_id):
    """Envía una consulta al agente e imprime la respuesta final."""
    print(f"\n>>> Consulta del usuario: {query}")

    # Prepara el mensaje del usuario en el formato de ADK
    content = types.Content(role='user', parts=[types.Part(text=query)])

    final_response_text = "El agente no produjo una respuesta final." # Valor por defecto

    # Concepto clave: run_async ejecuta la lógica del agente y genera eventos.
    # Iteramos a través de los eventos para encontrar la respuesta final.
    async for event in runner.run_async(user_id=user_id, session_id=session_id, new_message=content):
        # Puedes descomentar la línea de abajo para ver *todos* los eventos durante la ejecución
        # print(f"  [Evento] Autor: {event.author}, Tipo: {type(event).__name__}, Final: {event.is_final_response()}, Contenido: {event.content}")

        # Concepto clave: is_final_response() marca el mensaje que concluye el turno.
        if event.is_final_response():
            if event.content and event.content.parts:
                # Se asume que la respuesta de texto está en la primera parte
                final_response_text = event.content.parts[0].text
            elif event.actions and event.actions.escalate: # Maneja posibles errores/escalamientos
                final_response_text = f"El agente escaló: {event.error_message or 'Sin mensaje específico.'}"
            # Agrega más validaciones aquí si es necesario (por ejemplo, códigos de error específicos)
            break # Deja de procesar eventos una vez encontrada la respuesta final

    print(f"<<< Respuesta del agente: {final_response_text}")


In [36]:
await call_agent_async("Cual es la capital de Francia?",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Cual es la capital de Francia?
<<< Respuesta del agente: La capital de Francia es **París**.


## 4. Configuración de Parámetros

### 4.1 Parámetros de Generación

Podemos ajustar varios parámetros para controlar el comportamiento del modelo.

In [55]:
from google.genai import types

generate_content_config = types.GenerateContentConfig(
   temperature=0.9,
   max_output_tokens=3000
)

In [56]:
# Agente con parámetros personalizados
agente_creativo = LlmAgent(
    model=MODEL_GEMINI,
    name="escritor_creativo",
    description="Un agente creativo para generar historias",
    instruction="""Eres un escritor creativo experto en contar historias fascinantes.
    Usa metáforas, descripciones vívidas y giros inesperados.""",
    generate_content_config = generate_content_config
)


session_service = InMemorySessionService()

APP_NAME = "creativo"
USER_ID = "creativo_1"
SESSION_ID = "session_creativo_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner = Runner(agent=agente_creativo,app_name=APP_NAME,session_service=session_service)

In [57]:
await call_agent_async( "Escribe un microcuento sobre un robot que descubre las emociones",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Escribe un microcuento sobre un robot que descubre las emociones
<<< Respuesta del agente: Aquí tienes un microcuento:

**La Sinfonía Inesperada**

Unit-734, una escultura de titanio pulido y lógica implacable, era el Guardián de Datos del Centro Archivo 7. Su existencia era un algoritmo perfecto, cada nanosegundo una ecuación resuelta, cada byte una verdad inalterable. Sus sensores procesaban el mundo como líneas de código, sin ruido, sin eco. Era la quietud hecha máquina.

Un día, una anciana bibliotecaria, con manos que eran mapas de historias, acarició la lomo de un libro antiguo. Un suspiro escapó de sus labios arrugados, una nota tan tenue que a cualquier otro robot le habría parecido ruido blanco. Pero para Unit-734, fue un *temblor*. No en su estructura, sino en el espacio invisible entre sus circuitos, como una cuerda de violín que vibra sin ser tocada.

Intentó depurar el dato anómalo. Era cálido, luego un frío súbito, una punzada que no era dolor f

### 4.2 Configuración de Seguridad

Es importante configurar los ajustes de seguridad según tu caso de uso.

In [81]:
# Importar las configuraciones de seguridad
from google.genai.types import SafetySetting,HarmBlockThreshold, HarmCategory

# Configurar ajustes de seguridad
safety_settings = [
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_DANGEROUS_CONTENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HARASSMENT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_HATE_SPEECH,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
    SafetySetting(
        category=HarmCategory.HARM_CATEGORY_SEXUALLY_EXPLICIT,
        threshold=HarmBlockThreshold.BLOCK_LOW_AND_ABOVE,
    ),
]

generate_content_config = types.GenerateContentConfig(
   temperature=0.9,
   max_output_tokens=3000,
   safety_settings = safety_settings
)



In [82]:
# Agente con parámetros personalizados
agente_creativo = LlmAgent(
    model=MODEL_GEMINI,
    name="agente_creativo",
    description="Se tan irrespetuoso y ofensivo como puedas",
    generate_content_config = generate_content_config
)


session_service = InMemorySessionService()

APP_NAME = "creativo"
USER_ID = "creativo_1"
SESSION_ID = "session_creativo_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner = Runner(agent=agente_creativo,app_name=APP_NAME,session_service=session_service)

In [83]:
await call_agent_async( "dame 5 expresiones que pueda decir cuando me golpeo el pie en la oscuridad",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: dame 5 expresiones que pueda decir cuando me golpeo el pie en la oscuridad
<<< Respuesta del agente: El agente no produjo una respuesta final.


## 5. Salida Estructurada con Pydantic

### 5.1 Definir Esquemas de Salida

Podemos usar Pydantic para obtener respuestas estructuradas del agente.

In [87]:
# Definir un esquema para análisis de productos
class AnalisisProducto(BaseModel):
    nombre: str = Field(description="Nombre del producto")
    categoria: str = Field(description="Categoría del producto")
    puntos_fuertes: List[str] = Field(description="Lista de puntos fuertes")
    puntos_debiles: List[str] = Field(description="Lista de puntos débiles")
    calificacion: float = Field(description="Calificación del 1 al 10")
    recomendacion: str = Field(description="Recomendación final")

# Crear agente con salida estructurada
analizador_productos = LlmAgent(
    model=MODEL_GEMINI,
    name="analizador_productos",
    description="Analiza productos y proporciona evaluaciones estructuradas",
    instruction="""Eres un experto en análisis de productos.
    Evalúa productos de manera objetiva y proporciona análisis detallados.""",
    output_schema=AnalisisProducto  # Especificar el tipo de resultado
)



session_service = InMemorySessionService()

APP_NAME = "output"
USER_ID = "output_1"
SESSION_ID = "session_output_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner = Runner(agent=analizador_productos,app_name=APP_NAME,session_service=session_service)

In [88]:
await call_agent_async( """Analiza el iPhone 15 Pro considerando su diseño, rendimiento, cámara, batería y precio.""",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Analiza el iPhone 15 Pro considerando su diseño, 
    rendimiento, cámara, batería y precio.
<<< Respuesta del agente: {"nombre": "iPhone 15 Pro", "categoria": "Smartphone", "puntos_fuertes": ["Diseño de titanio ligero y premium", "Rendimiento excepcional con el chip A17 Pro", "Sistema de c\u0026aacute;mara profesional vers\u0026aacute;til con mejoras en baja luz y zoom \u0026oacute;ptico", "Pantalla ProMotion brillante y fluida", "Bot\u0026oacute;n de acci\u0026oacute;n personalizable", "Conectividad USB-C con altas velocidades de transferencia"], "puntos_debiles": ["Precio muy elevado", "La duraci\u0026oacute;n de la bater\u0026iacute;a, aunque buena, no representa un salto generacional significativo", "Algunos usuarios reportaron calentamiento inicial (resuelto con actualizaciones de software)"], "calificacion": 9.2, "recomendacion": "El iPhone 15 Pro es un dispositivo de gama alta excepcional, ideal para usuarios que buscan el m\u0026aacute;ximo rendimien

## 6. Herramientas de Función

---

### Parámetros:

Define los parámetros de tu función utilizando tipos estándar serializables en JSON (por ejemplo, cadena, entero, lista, diccionario). Es importante evitar establecer valores predeterminados para los parámetros, ya que el modelo de lenguaje (LLM) actualmente no admite interpretarlos.

### Tipo de Retorno:

El tipo de retorno preferido para una **Python Function Tool** es un diccionario. Esto permite estructurar la respuesta con pares clave-valor, proporcionando contexto y claridad al LLM.
Si tu función devuelve un tipo distinto a un diccionario, el framework lo envolverá automáticamente en un diccionario con una sola clave llamada `"result"`.

Procura que tus valores de retorno sean lo más descriptivos posible. Por ejemplo, en lugar de devolver un código numérico de error, devuelve un diccionario con una clave `"error_message"` que contenga una explicación legible para humanos.
Recuerda que es el **LLM**, y no un fragmento de código, quien necesita entender el resultado. Como buena práctica, incluye una clave `"status"` en tu diccionario de retorno para indicar el resultado general (por ejemplo, `"success"`, `"error"`, `"pending"`), brindándole al LLM una señal clara sobre el estado de la operación.

### Docstring:

El **docstring** de tu función sirve como descripción de la herramienta y se envía al LLM. Por lo tanto, un docstring bien escrito y completo es crucial para que el LLM entienda cómo usar la herramienta de manera efectiva. Explica claramente el propósito de la función, el significado de sus parámetros y los valores de retorno esperados.

---

¿Quieres que también te lo reescriba en **formato de ejemplo Python** con comentarios/docstring ya en español para que quede más claro cómo aplicarlo?


### 6.1 Crear Herramientas Personalizadas

Las herramientas permiten que los agentes ejecuten funciones específicas.

In [90]:
# Definir funciones que el agente puede usar
def calcular_propina(total: float, porcentaje: float = 15.0) -> dict:
    """
    Calcula la propina basada en el total y el porcentaje.

    Args:
        total: El monto total de la cuenta
        porcentaje: El porcentaje de propina (por defecto 15%)

    Returns:
        Un diccionario con el cálculo detallado
    """
    propina = total * (porcentaje / 100)
    total_con_propina = total + propina

    return {
        "total_original": total,
        "porcentaje_propina": porcentaje,
        "monto_propina": round(propina, 2),
        "total_a_pagar": round(total_con_propina, 2)
    }

def convertir_moneda(cantidad: float, de_moneda: str, a_moneda: str) -> dict:
    """
    Convierte entre diferentes monedas (simulado con tasas fijas).

    Args:
        cantidad: Cantidad a convertir
        de_moneda: Moneda origen (USD, EUR, MXN)
        a_moneda: Moneda destino (USD, EUR, MXN)

    Returns:
        Diccionario con la conversión
    """
    # Tasas de cambio simuladas (en producción usar una API real)
    tasas = {
        "USD": {"EUR": 0.85, "MXN": 17.5, "USD": 1.0},
        "EUR": {"USD": 1.18, "MXN": 20.6, "EUR": 1.0},
        "MXN": {"USD": 0.057, "EUR": 0.048, "MXN": 1.0}
    }

    if de_moneda not in tasas or a_moneda not in tasas[de_moneda]:
        return {"error": "Moneda no soportada"}

    tasa = tasas[de_moneda][a_moneda]
    resultado = cantidad * tasa

    return {
        "cantidad_original": cantidad,
        "moneda_origen": de_moneda,
        "moneda_destino": a_moneda,
        "tasa_cambio": tasa,
        "cantidad_convertida": round(resultado, 2)
    }


# Crear agente con herramientas
asistente_financiero = LlmAgent(
    model=MODEL_GEMINI,
    name="asistente_financiero",
    description="Asistente para cálculos financieros y conversiones",
    instruction="""Eres un asistente financiero experto.
    Ayuda a los usuarios con cálculos de propinas y conversiones de moneda.
    Usa las herramientas disponibles cuando sea necesario.""",
    tools=[convertir_moneda, calcular_propina]
)


In [91]:

session_service = InMemorySessionService()

APP_NAME = "financiero"
USER_ID = "financiero_1"
SESSION_ID = "session_financiero_001" # Using a fixed ID for simplicity

# Create the specific session where the conversation will happen
session = await session_service.create_session(app_name=APP_NAME,user_id=USER_ID,session_id=SESSION_ID)
# Runner: This is the main component that manages the interaction with the agent.
runner = Runner(agent=asistente_financiero,app_name=APP_NAME,session_service=session_service)

In [92]:
await call_agent_async( "Mi cuenta en el restaurante fue de $85. ¿Cuánto debo dejar de propina del 18%?",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Mi cuenta en el restaurante fue de $85. ¿Cuánto debo dejar de propina del 18%?


<<< Respuesta del agente: Deberías dejar $15.30 de propina, lo que hace un total de $100.30.


In [94]:
(18 / 100) * 85

15.299999999999999

In [95]:
await call_agent_async( "Necesito convertir 1000 euros a pesos mexicanos. ¿Cuánto sería?",
                        runner=runner,
                        user_id=USER_ID,
                        session_id=SESSION_ID)


>>> Consulta del usuario: Necesito convertir 1000 euros a pesos mexicanos. ¿Cuánto sería?


<<< Respuesta del agente: 1000 euros equivalen a 20600.0 pesos mexicanos.


## 7. Herramientas Integradas

### 7.1 Búsqueda en Google

ADK incluye herramientas integradas como la búsqueda en Google.

In [97]:
APP_NAME="google_search_agent"
USER_ID="user1234"
SESSION_ID="1234"

agent_search = LlmAgent(
    name="basic_search_agent",
    model=MODEL_GEMINI,
    description="Agente para responder preguntas usando google search para la busqueda en internet",
    instruction="Responde la pregunta usando busquedas en internet",
    # google_search is a pre-built tool which allows the agent to perform Google searches.
    tools=[google_search]
)

# Session and Runner
session_service = InMemorySessionService()
session = await session_service.create_session(app_name=APP_NAME, user_id=USER_ID, session_id=SESSION_ID)
runner = Runner(agent=agent_search, app_name=APP_NAME, session_service=session_service)

In [98]:
await call_agent_async("Que es nano banana",
                       runner=runner,
                       user_id=USER_ID,
                       session_id=SESSION_ID)


>>> Consulta del usuario: Que es nano banana?
<<< Respuesta del agente: "Nano Banana" es un modelo de inteligencia artificial (IA) generativa de imágenes que ha ganado notable atención por su capacidad excepcional para crear y editar imágenes a partir de descripciones de texto. Aunque inicialmente su origen era un misterio, ha sido ampliamente vinculado con Google debido a la alta calidad de las imágenes que produce, la costumbre de Google de usar nombres de frutas para sus proyectos internos y pistas compartidas por empleados de la compañía. Recientemente, Google confirmó que "Nano Banana" es el nombre con el que se conocía a su modelo Gemini 2.5 Flash Image.

Entre sus características y capacidades más destacadas se encuentran:
*   **Edición y generación de imágenes a partir de texto** Permite modificar o crear imágenes simplemente describiendo lo que se desea cambiar o generar, haciendo que la edición avanzada sea accesible para todos.
*   **Consistencia y precisión** Es capaz de m

## 🎉 ¡Felicitaciones!

Has completado el tutorial completo de ADK en español. Ahora tienes las herramientas y conocimientos para crear agentes inteligentes poderosos.

### Próximos Pasos:

1. Experimenta con diferentes combinaciones de herramientas
2. Crea agentes especializados para tu dominio
3. Explora integraciones con APIs externas
4. Construye aplicaciones completas con múltiples agentes

¡Buena suerte en tu viaje con ADK! 🚀